In [ ]:

import DataSetQuery
import AsyncDataSetQuery
import datetime
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime
from datetime import timedelta
from dateutil.relativedelta import *

import MalardHelpers as m

environmentName = 'JALOCALv3'
#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000',environmentName)

asyncquery = AsyncDataSetQuery.AsyncDataSetQuery( 'ws://localhost:9000',environmentName)


In [ ]:
query.getParentDataSets()

In [ ]:
query.getDataSets('mtngla')

In [ ]:
parentDs = 'mtngla'
dataSet = 'tdx3'
region = 'himalayas'

bbox = json.loads(query.getDataSetBoundingBox( parentDs,dataSet,region ))

In [ ]:
print(bbox)

In [ ]:
minX = bbox['gridCellMinX']
maxX = bbox['gridCellMaxX']
minY = bbox['gridCellMinY']
maxY = bbox['gridCellMaxY']
minT = datetime.datetime.fromtimestamp(bbox['minTime'])
maxT = datetime.datetime.fromtimestamp(bbox['maxTime'])

gridCellsDf = json_normalize(json.loads(query.getGridCells( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT )))


In [ ]:
shardsDf = json_normalize(json.loads(query.getShards( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT )))

print(shardsDf.info())

In [ ]:


def checkGridCellByMonth( gc ):
    count = 0 
    gcStartTime = datetime.datetime.fromtimestamp(gc['minTime'])
    gcEndTime = datetime.datetime.fromtimestamp(gc['maxTime'])
    minX = int(gc['gridCellMinX'])
    maxX = int(gc['gridCellMaxX'])
    minY = int(gc['gridCellMinY'])
    maxY = int(gc['gridCellMaxY'])
    
    adjStartTime = datetime.datetime( gcStartTime.year, gcStartTime.month, 1, 0, 0,0  )  
    incStartTime = adjStartTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    while adjStartTime < gcEndTime:
        minT = adjStartTime
        maxT = incStartTime
        
        cacheResult = asyncquery.executeQuery( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT,[],[] )
        
        if cacheResult.status != 'Error: Empty resultset.':
            df = m.getDataFrameFromNetCDF(cacheResult.resultFileName)
            count = count + len(df['x'])
            asyncquery.releaseCache(cacheResult.resultFileName)
        
        adjStartTime = incStartTime
        incStartTime = incStartTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    return count

def checkGridCell( gc ):
    count = 0 
    gcStartTime = datetime.datetime.fromtimestamp(gc['minTime'])
    gcEndTime = datetime.datetime.fromtimestamp(gc['maxTime'])
    minX = int(gc['gridCellMinX'])
    maxX = int(gc['gridCellMaxX'])
    minY = int(gc['gridCellMinY'])
    maxY = int(gc['gridCellMaxY'])
    minT = gcStartTime
    maxT = gcEndTime
        
    cacheResult = asyncquery.executeQuery( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT,['x'],[] )
        
    if cacheResult.status != 'Error: Empty resultset.':
        df = m.getDataFrameFromNetCDF(cacheResult.resultFileName)
        count = count + len(df['x'])
        asyncquery.releaseCache(cacheResult.resultFileName)
    else:
        print("Unexpected error... ")
        
    return count

In [ ]:
totalcount = 0

for rowId, gridCell  in gridCellsDf.iterrows():
    count = checkGridCellByMonth(gridCell)
    totalcount = count + totalcount
    if count != gridCell['totalPoints']:
        print("%d CumCount=%d MinX=%d MinY=%d QueryCount=%d GridCellCount=%d"%(rowId, totalcount, gridCell['gridCellMinX'], gridCell['gridCellMinY'],count, gridCell['totalPoints']))

print( 'Number of rows checked %d' % (totalcount))

In [ ]:
#MinX=-600000 MinY=300000 QueryCount=64074 GridCellCount=64077

MinX=-600000
MaxX = MinX + 100000
MinY=300000
MaxY=MinY+100000

shards = json_normalize(json.loads(query.getShards(parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, minT, maxT )))


In [ ]:
count = 0

for s in shards['shardName']:
    df = m.getDataFrameFromNetCDF(s)
    time = datetime.datetime.fromtimestamp(df['time'].min())
    startTime = datetime.datetime( time.year, time.month, 1, 0, 0,0  )  
    endTime = startTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    result = asyncquery.executeQuery( parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, startTime, endTime,[],[]   )
    dfRes = m.getDataFrameFromNetCDF( result.resultFileName )
    
    print( 'StartTime %s EndTime %s ShardCount %d QueryCount %d' % (startTime, endTime, len(df['x']), len(dfRes['x'])))
    
    count = count + len(df['x'])
    
print(count)

In [ ]:
maxT = datetime.datetime(2019,12,30,23,59,59)
minT = datetime.datetime(2012,1,1,0,0,0)

result = asyncquery.executeQuery( parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, minT, maxT,[],[]   )

dfRes = m.getDataFrameFromNetCDF( result.resultFileName )

print(len(dfRes['x'])  ) 

dfRes.to_csv("first.csv")